In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np
import keras.utils.np_utils as kutils
from sklearn.model_selection import train_test_split
import keras.callbacks as callbacks
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.utils import plot_model
from keras.optimizers import SGD

Using TensorFlow backend.


In [3]:
num_classes = 4

num_reps = 10

In [4]:
import random
random.seed(123)
import os

In [5]:
#Load the data.py
airplane = np.load('airplane.npy')
axe = np.load('axe.npy')
bed = np.load('bed.npy')
bicycle = np.load('bicycle.npy')
butterfly = np.load('butterfly.npy')
envelope = np.load('envelope.npy')
knife = np.load('knife.npy')
square = np.load('square.npy')
star = np.load('star.npy')
donut = np.load('donut.npy')
classes = [airplane, axe, bed, bicycle, butterfly, envelope, knife, square, star, donut]

In [6]:
batch_size = 128
epochs = 12

In [7]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(28,28,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(num_classes, activation='softmax'))

In [9]:
callbacks = [ModelCheckpoint('weights',
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                            mode='auto')
            ]
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["acc"])

In [ ]:
test_eval = [None]*num_reps
used_classes = [None]*num_reps

for r in range(num_reps):
    random_classes = sorted(random.sample(range(0, 9), num_classes))
    used_classes[r] = random_classes
    for i,j in enumerate(random_classes):
        random_classes[i] = np.c_[classes[j], np.ones(classes[j].shape[0])*i]
    Data = np.concatenate(tuple(random_classes),axis=0)
    X = Data[:,0:784]
    Y = Data[:,784]
    del Data

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
    del X, Y
    X_train = X_train.reshape(-1, 28,28, 1)
    X_test = X_test.reshape(-1,28,28,1)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train = X_train / 255.
    X_test = X_test / 255.
    y_train = kutils.to_categorical(Y_train)
    y_test = kutils.to_categorical(Y_test)
    X_train,X_valid,y_train,y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=13)

    model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs,
                       callbacks=callbacks,
                       validation_data=(X_valid, y_valid))

    test_eval[r] = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss (Loop ', r, '): ', test_eval[r][0])
    print('Test accuracy (Loop ', r, '): ', test_eval[r][1])
    
    os.remove('weights')

Train on 412562 samples, validate on 45841 samples
Epoch 1/12
412562/412562 [==============================] - 35s 85us/step - loss: 0.0757 - acc: 0.9761 - val_loss: 0.0611 - val_acc: 0.9791

Epoch 00001: val_acc did not improve
Epoch 2/12
412562/412562 [==============================] - 39s 95us/step - loss: 0.0480 - acc: 0.9837 - val_loss: 0.0572 - val_acc: 0.9801

Epoch 00002: val_acc did not improve
Epoch 3/12
412562/412562 [==============================] - 35s 85us/step - loss: 0.0375 - acc: 0.9868 - val_loss: 0.0620 - val_acc: 0.9796

Epoch 00003: val_acc did not improve
Epoch 4/12
 98560/412562 [======>.......................] - ETA: 25s - loss: 0.0257 - acc: 0.9910

In [ ]:
Loss = np.array([test_eval1[0],test_eval2[0],test_eval3[0]])
Accuracy = np.array([test_eval1[1],test_eval2[1],test_eval3[1]])
print('Average Test loss:', np.mean(Loss))
print('Average Test accuracy:', np.mean(Accuracy))